In [ ]:
# %% [markdown]
# ## Binding Module Figures
# 
# This section generates illustrative plots for the biorecognition module showing
# aptamer binding kinetics and binding noise characteristics.

# %%
# Import binding module functions
from src.binding import (
    bernoulli_binding,
    mean_binding,
    binding_noise_psd
)
from scipy import signal

import numpy as np
import matplotlib.pyplot as plt
import yaml, pathlib

cfg_file = pathlib.Path("../config/default.yaml")          # adjust path if notebook lives elsewhere
with cfg_file.open("r") as f:
    config = yaml.safe_load(f)

# Convert string values to float where needed
def convert_string_numbers(config_dict):
    """Recursively convert string numbers to floats in config dictionary"""
    for key, value in config_dict.items():
        if isinstance(value, dict):
            convert_string_numbers(value)
        elif isinstance(value, str):
            try:
                # Try to convert scientific notation strings to float
                config_dict[key] = float(value)
            except ValueError:
                # Keep as string if conversion fails
                pass
    return config_dict

config = convert_string_numbers(config)

# --- START OF THE NEW CODE BLOCK YOU ARE ADDING ---

# Create the nested dictionaries that our refactored functions expect
# This ensures this notebook is compatible with the latest src code.
config['oect'] = {}
config['sim'] = {}

# Move the relevant parameters from the top level into the new nested structure
config['oect']['gm_S'] = config.get('gm_S', 0.002)
config['oect']['C_tot_F'] = config.get('C_tot_F', 2.4e-7) # Using the latest tuned value

config['sim']['dt_s'] = config.get('dt_s', 0.01)

# --- END OF THE NEW CODE BLOCK ---

# %% [markdown]
# ### Figure 3a: Binding Kinetics - Deterministic vs Monte Carlo
# 
# Compare the deterministic mean-field solution with stochastic Monte Carlo traces
# to illustrate the shot noise inherent in aptamer binding.

# %%
# Setup parameters for binding kinetics
C_eq = 10e-9  # 10 nM constant concentration
nt_type = 'GLU'
duration = 10.0  # seconds
dt = config['dt_s']
n_steps = int(duration / dt)

# Create constant concentration profile
conc_time = np.full(n_steps, C_eq)
t_vec = np.arange(n_steps) * dt

# Calculate deterministic mean binding
N_b_mean = mean_binding(conc_time, nt_type, config)

# Generate 3 Monte Carlo traces with different seeds
rng = np.random.default_rng(2025)
mc_traces = []
seeds = [42, 123, 456]
colors = ['blue', 'red', 'green']

for seed in seeds:
    rng_mc = np.random.default_rng(seed)
    bound_sites, _, _ = bernoulli_binding(conc_time, nt_type, config, rng_mc)
    mc_traces.append(bound_sites)

# Create the plot
fig, ax = plt.subplots(figsize=(10, 6))

# Plot deterministic mean (thick black line)
ax.plot(t_vec, N_b_mean / 1e6, 'k-', linewidth=3, label='Deterministic mean', zorder=10)

# Plot Monte Carlo traces (semi-transparent colored lines)
for i, (trace, color) in enumerate(zip(mc_traces, colors)):
    ax.plot(t_vec, trace / 1e6, color=color, alpha=0.4, linewidth=1.5, 
            label=f'MC trace {i+1}')

ax.set_xlabel('Time (s)', fontsize=14)
ax.set_ylabel(f'Bound sites (×10⁶, μ≈{N_b_mean[-1]/1e6:.3f})', fontsize=14)
ax.set_title('Binding Kinetics: Deterministic vs Monte Carlo', fontsize=16)
ax.grid(True, alpha=0.3)
ax.legend(loc='center right', fontsize=12)
ax.set_xlim(0, 10)

plt.tight_layout()
plt.savefig('../results/figures/binding_mean_vs_mc.png', dpi=300)
plt.show()

print(f"Equilibrium binding level: {N_b_mean[-1]/1e6:.2f} × 10⁶ sites")
fluct = np.std(np.concatenate([trace[-100:] for trace in mc_traces]))
print(f"Relative fluctuations: ±{fluct/N_b_mean[-1]*100:.1f}%")

# %% [markdown]
# ### Figure 3b: Binding Noise PSD vs Analytical Lorentzian
# 
# Compare the empirical power spectral density from Monte Carlo simulation
# with the analytical Lorentzian formula to validate the noise model.

# %%
# Generate long time series for good PSD estimate
duration_psd = 100.0  # seconds for better low-frequency resolution
n_steps_psd = int(duration_psd / dt)
conc_time_psd = np.full(n_steps_psd, C_eq)

# Run Monte Carlo simulation and get AC component
rng_psd = np.random.default_rng(789)
_, _, ibind_ac = bernoulli_binding(conc_time_psd, nt_type, config, rng_psd)

# Calculate Welch PSD
fs = 1 / dt
nperseg = min(16384, n_steps_psd // 4)  # Use 16384 or 1/4 of signal length
f_welch, psd_welch = signal.welch(
    ibind_ac,
    fs=fs,
    nperseg=nperseg,
    noverlap=nperseg // 2,
    scaling='density',
    window='hann'
)

# Calculate analytical PSD
# Only use frequencies where Welch PSD is valid (> 0 Hz)
valid_mask = f_welch > 0
f_analytic = f_welch[valid_mask]
psd_analytic = binding_noise_psd(nt_type, config, f_analytic, C_eq)

# Create log-log plot
fig, ax = plt.subplots(figsize=(10, 7))

# Plot Welch estimate (blue line)
ax.loglog(f_welch[valid_mask], psd_welch[valid_mask], 'b-', 
          linewidth=2, label='Welch estimate', alpha=0.8)

# Plot analytical PSD (dashed black line)
ax.loglog(f_analytic, psd_analytic, 'k--', 
          linewidth=2, label='Analytical Lorentzian')

# Set axis limits and labels
ax.set_xlim(0.1, 50)
ax.set_xlabel('Frequency (Hz)', fontsize=14)
ax.set_ylabel('PSD (A²/Hz)', fontsize=14)
ax.set_title('Binding Noise PSD: Empirical vs Analytical', fontsize=16)
ax.grid(True, which='both', alpha=0.3)
ax.legend(fontsize=12)

# Add characteristic frequency annotation
from src.binding import calculate_equilibrium_metrics
metrics = calculate_equilibrium_metrics(C_eq, nt_type, config)
f_c = 1 / (2 * np.pi * metrics['tau_B'])
ax.axvline(f_c, color='gray', linestyle=':', alpha=0.7)
ylims = ax.get_ylim()
ax.text(f_c * 1.2, ylims[1]*0.3,  # place at 30 % of full scale
        f'$f_c$ = {f_c:.1f} Hz', rotation=90,
        va='bottom', ha='left', fontsize=10)

plt.tight_layout()
plt.savefig('../results/figures/binding_psd_vs_analytic.png', dpi=300)
plt.show()

print(f"Binding relaxation time τ_B: {metrics['tau_B']*1000:.1f} ms")
print(f"Corner frequency f_c: {f_c:.1f} Hz")
print(f"Expected current noise σ_I: {metrics['std_current']*1e12:.1f} pA")

# %% [markdown]
# ### Summary of Binding Module Results
# 
# 1. **Binding Kinetics**: The Monte Carlo traces show realistic shot noise around the 
#    deterministic mean, with fluctuations proportional to √N as expected from binomial statistics.
# 
# 2. **Noise PSD**: The empirical PSD from Monte Carlo simulation matches the analytical 
#    Lorentzian formula well, validating our noise model. The corner frequency corresponds 
#    to the binding relaxation time τ_B.
# 
# These results confirm that the binding module correctly implements both the mean-field 
# dynamics and the stochastic fluctuations described in Section II-E of the manuscript.